In [1]:
# Colab: mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# =========================
# SHAP (V2) — LSTM, TinyFormer, LiPFormer, LiteFormer
# Robust to keras/savedmodel loaders, dict outputs, multi-horizon shapes.
# Produces: time×feature heatmap, feature-family bar, beeswarm (top-K).
# =========================
import os, json, warnings
import numpy as np
import matplotlib.pyplot as plt
import shap
import tensorflow as tf
from tensorflow import keras

# -------------------------
# CONFIG (paths & knobs)
# -------------------------
V2_DIR   = "/content/drive/MyDrive/EdgeMeter_AIv2/data"
OUT_ROOT = os.path.join(V2_DIR, "Model_Comparison_Final", "shap_v2")
os.makedirs(OUT_ROOT, exist_ok=True)

# Expected test arrays
X_TEST_PATH = os.path.join(V2_DIR, "X_test_48_12.npy")
Y_TEST_PATH = os.path.join(V2_DIR, "y_test_48_12.npy")

# Model basenames (we try .keras → .h5 → SavedModel folder)
V2_BASENAMES = {
    "LSTM"      : "Final_LSTM_Model_48_12",
    "TinyFormer": "Final_TinyFormer_Model_48_12",
    "LiPFormer" : "Final_LiPFormer_Model_48_12",
    "LiteFormer": "Final_LiteFormer_Model_48_12",
}

# SHAP sampling budget (balance speed vs fidelity)
N_BACKGROUND = 50      # background examples for KernelExplainer
N_SAMPLES    = 50      # test examples to explain
TOPK_BEE     = 30      # top-K flattened features to show in beeswarm

# Random seed for reproducibility
RNG = np.random.default_rng(42)

# -------------------------
# Feature names (best-effort)
# We will try to load JSON/TXT names; else fallback to plausible/default.
# -------------------------
def load_feature_names(F: int):
    # candidates to look for in V2_DIR
    candidates = [
        os.path.join(V2_DIR, "feature_names_48_12.json"),
        os.path.join(V2_DIR, "feature_names_v2.json"),
        os.path.join(V2_DIR, "feature_names.json"),
        os.path.join(V2_DIR, "feature_names.txt"),
    ]
    for p in candidates:
        if os.path.exists(p):
            try:
                if p.endswith(".json"):
                    with open(p, "r") as f:
                        names = json.load(f)
                else:
                    with open(p, "r") as f:
                        names = [line.strip() for line in f if line.strip()]
                if isinstance(names, dict):      # handle dict formats
                    names = list(names.values()) if "names" not in names else names["names"]
                names = list(names)
                if len(names) == F:
                    print(f"[info] Loaded feature names from: {os.path.basename(p)}")
                    return names
            except Exception as e:
                print(f"[warn] Failed to parse {p}: {e}")

    # plausible fallback for 11‑feature V2 (tweak if you have a definitive list)
    plausible_11 = [
        "consumption",
        "is_uk_bank_holiday",
        "day_of_week",
        "is_weekend",
        "month",
        "day",
        "hour",
        "minute",
        "is_friday",
        "is_christmas_week",
        "usage_flag",  # fallback label (sometimes 'usage_bin' or 'is_halloween_week' in your variants)
    ]
    if F == 11:
        print("[warn] Using fallback V2 feature names (11). Consider supplying a JSON list for exact naming.")
        return plausible_11

    # generic fallback
    print("[warn] Using generic feature names.")
    return [f"f{i}" for i in range(F)]

# -------------------------
# Loaders (keras/h5/SavedModel)
# -------------------------
class SavedModelRunner:
    """Run a TF SavedModel signature directly (no Keras deserialization)."""
    def __init__(self, savedmodel_dir: str):
        obj = tf.saved_model.load(savedmodel_dir)
        fn = obj.signatures.get("serving_default")
        if fn is None:
            sigs = list(obj.signatures.keys())
            if not sigs:
                raise ValueError(f"No signatures in SavedModel: {savedmodel_dir}")
            fn = obj.signatures[sigs[0]]
        self.fn = fn
        self._input_keys = list(self.fn.structured_input_signature[1].keys())
        self._output_keys = list(self.fn.structured_outputs.keys())
    def predict(self, x):
        """Return numpy predictions; do NOT accept verbose kwarg."""
        x = tf.convert_to_tensor(x)
        out = self.fn(**{self._input_keys[0]: x})
        return out[self._output_keys[0]].numpy()

def _try_load_any(base_dir, basename):
    p_keras = os.path.join(base_dir, basename + ".keras")
    p_h5    = os.path.join(base_dir, basename + ".h5")
    p_dir   = os.path.join(base_dir, basename)
    if os.path.exists(p_keras):
        try:
            return keras.models.load_model(p_keras, compile=False)
        except Exception as e:
            print(f"[warn] .keras load failed for {basename}: {e}")
    if os.path.exists(p_h5):
        try:
            return keras.models.load_model(p_h5, compile=False)
        except Exception as e:
            print(f"[warn] .h5 load failed for {basename}: {e}")
    if os.path.isdir(p_dir):
        print(f"[info] Using SavedModelRunner for: {basename}")
        return SavedModelRunner(p_dir)
    raise FileNotFoundError(f"No loadable artifact for {basename} in {base_dir}")

def load_models(base_dir, basenames):
    return {name: _try_load_any(base_dir, base) for name, base in basenames.items()}

# -------------------------
# Robust prediction helper for SHAP
# -------------------------
def _coerce_first_step_1d(y):
    """Return 1-D first-step predictions from varied model outputs."""
    if isinstance(y, dict):
        y = next(iter(y.values()))
    y = np.asarray(y)
    # Shapes we might see: (N, 12), (N, 12, 1), (N, 1), (N,)
    if y.ndim == 3:
        y = y[:, 0]          # (N, 1) or (N,)
        if y.ndim == 2 and y.shape[1] == 1:
            y = y[:, 0]
    elif y.ndim == 2:
        y = y[:, 0]
    return y.astype(np.float32)

def model_predict_first_step(model_obj, X):
    """Call model predict robustly and return 1-D first-step preds."""
    try:
        y = model_obj.predict(X, verbose=0)   # works for Keras models
    except TypeError:
        y = model_obj.predict(X)              # SavedModelRunner has no verbose
    except Exception:
        y = model_obj.predict(X)              # last resort
    return _coerce_first_step_1d(y)

def make_predict_fn(model_obj, F):
    """Flattened → (N, 48, F) → 1D first-step preds."""
    def predict_fn(x_flat):
        x = x_flat.reshape((-1, 48, F)).astype(np.float32)
        # device fallback if needed (rare with KernelExplainer's CPU path)
        try:
            return model_predict_first_step(model_obj, x)
        except tf.errors.InternalError as e:
            if "CudnnRNN" in str(e) or "DoRnnForward" in str(e) or "CUDNN" in str(e):
                with tf.device("/CPU:0"):
                    return model_predict_first_step(model_obj, x)
            raise
    return predict_fn

# -------------------------
# SHAP array normalization (handles list/array variants, multi-output)
# -------------------------
def normalize_shap_array(shap_out, S_expected, M_expected):
    """
    Returns (S_expected, M_expected) array.
    Accepts:
      - np.ndarray shape (S,M) or (M,S) or (S,M,1) …
      - list of arrays per output; we take the first horizon.
    """
    if isinstance(shap_out, (list, tuple)):
        arr = np.asarray(shap_out[0])
    else:
        arr = np.asarray(shap_out)

    # squeeze singleton dims, then try to coerce
    arr = np.squeeze(arr)

    # Straight fit
    if arr.shape == (S_expected, M_expected):
        return arr

    # Transposed (M, S)
    if arr.shape == (M_expected, S_expected):
        return arr.T

    # Sometimes SHAP returns (S, M, 1)
    if arr.ndim == 3 and arr.shape[2] == 1 and arr.shape[0] == S_expected and arr.shape[1] == M_expected:
        return arr[:, :, 0]

    # Last resort: flatten and reshape with correct total size
    flat = arr.reshape(S_expected, -1)
    if flat.shape[1] == M_expected:
        return flat

    raise ValueError(f"Could not coerce SHAP array with shape {arr.shape} to (S={S_expected}, M={M_expected}).")

# -------------------------
# Plot helpers
# -------------------------
def _ensure_2d(a):
    a = np.asarray(a)
    return a if a.ndim == 2 else a.reshape(a.shape[0], -1)

def plot_feature_family_bar(shap_vals, feat_names, F, out_png):
    """
    Aggregate |SHAP| across 48 time steps -> one value per feature family.
    """
    vals = np.abs(_ensure_2d(shap_vals)).mean(axis=0)      # (48*F,)
    agg  = np.zeros(F, dtype=np.float64)
    for f in range(F):
        cols = np.arange(f, 48*F, F)                       # f at all 48 steps
        agg[f] = vals[cols].mean()

    order = np.argsort(agg)[::-1]
    plt.figure(figsize=(8, 4))
    plt.bar(np.arange(F), agg[order])
    plt.xticks(np.arange(F), [feat_names[i] for i in order], rotation=45, ha="right")
    plt.ylabel("Mean |SHAP| (aggregated across time)")
    plt.title("SHAP — Feature family importance (V2)")
    plt.tight_layout()
    plt.savefig(out_png, dpi=220, bbox_inches="tight")
    plt.close()

def plot_time_feature_heatmap(shap_vals, feat_names, F, out_png):
    """
    Heatmap of mean |SHAP| over samples: 48 × F (time rows × features cols)
    """
    vals = np.abs(_ensure_2d(shap_vals)).mean(axis=0)      # (48*F,)
    M    = vals.reshape(48, F)
    plt.figure(figsize=(10, 3.8))
    plt.imshow(M, aspect="auto", origin="lower")
    plt.colorbar(label="Mean |SHAP|")
    plt.yticks(np.arange(0, 49, 6), [f"t-{48-i}" for i in np.arange(0, 49, 6)])
    plt.xticks(np.arange(F), feat_names, rotation=45, ha="right")
    plt.title("SHAP — Time × Feature importance (V2)")
    plt.tight_layout()
    plt.savefig(out_png, dpi=220, bbox_inches="tight")
    plt.close()

def plot_beeswarm_topk(shap_vals, feat_names, F, out_png, topk=30):
    """
    Beeswarm on flattened features. Build names like `feat@t-7`.
    """
    S = shap_vals.shape[0]
    # Build per-position names
    fl_names = []
    for t in range(48):
        for f in range(F):
            fl_names.append(f"{feat_names[f]}@t-{48 - t}")

    vals = _ensure_2d(shap_vals)  # (S, 48*F)
    # Rank by mean |SHAP|
    mean_abs = np.abs(vals).mean(axis=0)
    top_idx  = np.argsort(mean_abs)[::-1][:min(topk, mean_abs.size)]
    shap.summary_plot(vals[:, top_idx], feature_names=[fl_names[i] for i in top_idx],
                      show=False, plot_type="dot", color=None)  # default colormap
    plt.title("SHAP — Beeswarm (top features across time)")
    plt.tight_layout()
    plt.savefig(out_png, dpi=220, bbox_inches="tight")
    plt.close()

# -------------------------
# Load data
# -------------------------
if not os.path.exists(X_TEST_PATH) or not os.path.exists(Y_TEST_PATH):
    raise FileNotFoundError("X_test_48_12.npy / y_test_48_12.npy not found in V2_DIR.")

X = np.load(X_TEST_PATH).astype(np.float32)   # (N, 48, F)
y = np.load(Y_TEST_PATH).astype(np.float32)   # (N, 12)
assert X.ndim == 3 and X.shape[1] == 48, f"Unexpected X shape: {X.shape}"
N, T, F = X.shape
assert y.ndim == 2 and y.shape[1] >= 1, f"Unexpected y shape: {y.shape}"

feat_names = load_feature_names(F)

# Select background & sample indices
all_idx = np.arange(N)
bg_idx  = RNG.choice(all_idx, size=min(N_BACKGROUND, N), replace=False)
sm_idx  = RNG.choice(all_idx, size=min(N_SAMPLES, N),    replace=False)

background = X[bg_idx]
X_sample   = X[sm_idx]

# -------------------------
# Load models
# -------------------------
models_v2 = load_models(V2_DIR, V2_BASENAMES)

# -------------------------
# Run SHAP per model
# -------------------------
warnings.filterwarnings("ignore", category=UserWarning)
for model_name in ["LSTM", "TinyFormer", "LiPFormer", "LiteFormer"]:
    print(f"\n=== SHAP (V2) for {model_name} ===")
    model  = models_v2[model_name]
    outdir = os.path.join(OUT_ROOT, model_name)
    os.makedirs(outdir, exist_ok=True)

    # SHAP explainer configured with our safe predict function
    predict_fn = make_predict_fn(model, F)
    explainer  = shap.KernelExplainer(predict_fn, background.reshape(len(background), -1))
    shap_raw   = explainer.shap_values(X_sample.reshape(len(X_sample), -1))  # may be list/array

    # Normalize to (S, 48*F)
    shap_vals = normalize_shap_array(shap_raw, S_expected=X_sample.shape[0], M_expected=48*F)

    # Save raw SHAP for reuse
    np.save(os.path.join(outdir, "shap_values_V2_firststep.npy"), shap_vals)

    # 1) Feature family bar (aggregated across time)
    plot_feature_family_bar(
        shap_vals, feat_names, F,
        out_png=os.path.join(outdir, "shap_family_bar.png")
    )

    # 2) Time × Feature heatmap
    plot_time_feature_heatmap(
        shap_vals, feat_names, F,
        out_png=os.path.join(outdir, "shap_heatmap_time_by_feature.png")
    )

    # 3) Beeswarm (top-K flattened features)
    plot_beeswarm_topk(
        shap_vals, feat_names, F,
        out_png=os.path.join(outdir, "shap_beeswarm_topk.png"),
        topk=TOPK_BEE
    )

    print(f"[OK] SHAP plots saved → {outdir}")

print("\n[DONE] SHAP (V2) completed for all models.")


[warn] Using fallback V2 feature names (11). Consider supplying a JSON list for exact naming.
[warn] .keras load failed for Final_TinyFormer_Model_48_12: The `function` of this `Lambda` layer is a Python lambda. Deserializing it is unsafe. If you trust the source of the config artifact, you can override this error by passing `safe_mode=False` to `from_config()`, or calling `keras.config.enable_unsafe_deserialization().
[warn] .h5 load failed for Final_TinyFormer_Model_48_12: Exception encountered when calling Lambda.call().

We could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.

Arguments received by Lambda.call():
  • args=('<KerasTensor shape=(None, 48, 11), dtype=float32, sparse=False, ragged=False, name=input_layer>',)
  • kwargs={'mask': 'None'}
[info] Using SavedModelRunner for: Final_TinyFormer_Model_48_12
[warn] .keras load failed for Final_LiPFormer_Model_48_12: The `function` of this `Lambda` layer

  0%|          | 0/50 [00:00<?, ?it/s]

[OK] SHAP plots saved → /content/drive/MyDrive/EdgeMeter_AIv2/data/Model_Comparison_Final/shap_v2/LSTM

=== SHAP (V2) for TinyFormer ===


  0%|          | 0/50 [00:00<?, ?it/s]

[OK] SHAP plots saved → /content/drive/MyDrive/EdgeMeter_AIv2/data/Model_Comparison_Final/shap_v2/TinyFormer

=== SHAP (V2) for LiPFormer ===


  0%|          | 0/50 [00:00<?, ?it/s]

[OK] SHAP plots saved → /content/drive/MyDrive/EdgeMeter_AIv2/data/Model_Comparison_Final/shap_v2/LiPFormer

=== SHAP (V2) for LiteFormer ===


  0%|          | 0/50 [00:00<?, ?it/s]

[OK] SHAP plots saved → /content/drive/MyDrive/EdgeMeter_AIv2/data/Model_Comparison_Final/shap_v2/LiteFormer

[DONE] SHAP (V2) completed for all models.


In [ ]:
# This command forces Colab runtime to disconnect/terminate
os.kill(os.getpid(), 9)